This notebook will demonstrate a simple compile and deployment workflow using a ResNet50 pre-trained image recognizer with sample input images

First, we define all of our parameters that will be used in the notebook

In [ ]:
# Setup our Sagemaker configuration tunables
aws_region = 'us-east-1'
aws_s3_folder = "DEMO-Sagemaker-Edge"

# We will use the ResNet50 image recognition model via Keras framework
model_framework = 'keras'
model_basename = 'resnet50'
model_name = model_framework + "-" + model_basename + "-image-recognizer"
packaged_model_name = model_framework + "-" + model_basename + "-model"
packaged_model_version = "1.0"

# Input images directory in our notebook...
image_paths = './images'

# Input images WxH is 224x224 and are color images (so 3 channels)
image_size_w       = 224
image_size_h       = 224
image_num_channels = 3

# Channel first option
channels_first = True

# Our Pelion Edge Gateway is a Nvidia Jetson Xavier
target_device = 'jetson_xavier'

# Set our Pelion API Configuration Here
api_key = 'INSERT_YOUR_PELION_APPLICATION_KEY_HERE'
device_id = 'INSERT_YOUR_PELION_XAVIER_EDGE_SAGE_PT_DEVICEID_HERE' # Pelion Device ID of our Sagemaker Edge Agent PT device under our Pelion Edge gateway
endpoint_api = 'api.' + aws_region + '.mbedcloud.com'              # This is optional and the default

Next we import some additional packages into our python environment... including the pelion/sagemaker controller package

In [ ]:
# Lets install some image utilities
! pip install ipympl

# Core imports for the notebook
import numpy as np 
import pandas as pd
import os
import json
import tensorflow as tf
import base64

# Channels First modification for Keras
if channels_first == True:
    # Neo compiles require channel_first usage
    from tensorflow.keras import backend

    # force channels-first ordering
    backend.set_image_data_format('channels_first')

# We will use the ResNet50 Keras model, pre-trained with imagenet weights...
from tensorflow.keras.applications import ResNet50

# ResNet50 helpers used specifically with our selected pre-trained model
from tensorflow.keras.applications.resnet50 import preprocess_input

# We'll also use time for waiting on predictions to complete...
import time

# we will also use some helpers from numpy...
from numpy import asarray
from numpy import save

# Helper for debugging this notebook
import importlib

# We also need to install the Pelion Sagemaker Controller API
! pip install pelion_sagemaker_controller

# Method to Decode our ResNet50 based predictions
def decode_resnet50_predictions(preds, top=5, class_list_path=None):
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                     'a batch of predictions '
                     '(i.e. a 2D array of shape (samples, 1000)). '
                     'Found array with shape: ' + str(preds.shape))
    class_index = json.load(open(class_list_path))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(class_index[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

Next we import (or uncomment to re-import) the core class for this notebook

In [ ]:
import example_notebook
# importlib.reload(example_notebook)

Next we allocate our notebook class... this will init both Sagamaker as well as the Pelion Controller API

In [ ]:
my_notebook = example_notebook.MyNotebook(api_key, device_id, endpoint_api, aws_s3_folder)

Now... next we prepare our input images and upload them to S3 as preprocessed images for ResNet50 from Keras...

In [ ]:
# collect and prepare our images to analyze
print("")
print("Collecting images from: " + str(image_paths))
my_image_list = [os.path.join(image_paths,filename) for filename in os.listdir(image_paths) if os.path.isfile(image_paths + '/' + filename)]
my_image_list_length = len(my_image_list)

print("")
print("Input Image Files: " + json.dumps(my_image_list))

# preprocess_input() is ResNet50 specific
print("")
print("Pre-processing input image files...")
image_data = my_notebook.read_and_prep_images(my_image_list, image_size_h, image_size_w, channels_first)
preprocessed_images_list = preprocess_input(image_data['array'])

# Display our initial images
print("Displaying Input Images:")
my_notebook.display_images(image_data['img'])

# Neo Sagemaker likes NCHW. Our image set is in NHWC... so we must convert it...
print("")
print("Current input shape: " + str(preprocessed_images_list.shape))

# Reshape to NHWC -> NCHW if not channels_first already...
nchw_preprocessed_images_list = preprocessed_images_list
if channels_first == False:
    nchw_preprocessed_images_list = tf.transpose(preprocessed_images_list, [0, 3, 1, 2])
    print("")
    print("Transposed Input shape: " + str(nchw_preprocessed_images_list.shape))
    print("Transposed Input dtype: " + str(nchw_preprocessed_images_list.dtype))
    print("Transposed Input data length number of images: " + str(len(nchw_preprocessed_images_list)))
else:
    print("")
    print("Input shape: " + str(nchw_preprocessed_images_list.shape))
    print("Input dtype: " + str(nchw_preprocessed_images_list.dtype))
    print("Input data length number of images: " + str(len(nchw_preprocessed_images_list)))

# Save the preprocessed images_list as a single input file locally... 
input_data_filename = 'preprocessed_images_' + str(time.time()) + '.input'
input_data_filename_saved = input_data_filename + '.npy'

# save to a numpy-compatible file
save(input_data_filename, nchw_preprocessed_images_list)

# DEBUG
#print("")
#t = tf.reshape(nchw_preprocessed_images_list,[-1])
#t_npy = t.numpy()
#print(t_npy[0:1000])

# Upload the images in the list to S3
print("")
print('Uploading preprocessed input images to ' + my_notebook.iot_folder + " in S3 bucket " + my_notebook.bucket + ' as: ' + input_data_filename_saved + "...")
print("")
my_notebook.sess.upload_data(input_data_filename_saved, my_notebook.bucket, my_notebook.iot_folder)

Next, lets import a pre-trained Mobilenet v2 model via TF Hub...

In [ ]:
N = my_image_list_length
C = image_num_channels
H = image_size_h
W = image_size_w
resnet50_model = []

# Create the ResNet50 pre-trained model via Keras framework... channels_first shapes (CHW)...
if channels_first == True:
    print("Allocating ResNet50 with channels_first shape")
    resnet50_model = ResNet50(weights="imagenet")
else:
    print("Allocating ResNet50 with channels_last shape")
    resnet50_model = ResNet50(weights="imagenet")
    
# Lets dump the model details... 
print(resnet50_model.summary())

Next, we compile up the model and package it for sending down to the edge agent on the gateway...

In [ ]:
# Next, lets record the input layer for the Neo-compatible input_shape for Keras-based models...
input_layer = resnet50_model.get_layer(index=0)

# Neo wants me to build out a specific input_shape for the compile() task...
neo_input_layer_shape = {}
neo_input_layer_shape[input_layer.name] = [N, C, H, W]
neo_input_layer_shape = json.dumps(neo_input_layer_shape)
print("")
print("Neo expects this input_shape if compiling a Keras model: " + neo_input_layer_shape)

# Compile up for our target Pelion Edge Gateway platform type
print("")
print("Initiating Sagemaker Neo compile of " + model_name + "...")
job_name = my_notebook.compile_model(resnet50_model, target_device, model_basename, model_framework, neo_input_layer_shape)

# Package up and store the compiled model onto S3
print("")
print("Packaging up Neo-compiled model and placing in S3...")
model_package = my_notebook.package_model(packaged_model_name, packaged_model_version, job_name)

Next, we (re)load our model since we have just (re)compiled it and (re)packaged it...

In [ ]:
# (re)load the model...
print('Reloading Model: ' + model_name + " using package: " + model_package + '...')
print("")
my_notebook.pelion_api.pelion_reload_model(model_name,model_package)

# Poll every 5 sec to look for the reload() completion....
while True:
    print("Reloading " + model_name + "...")
    is_running = my_notebook.pelion_api.pelion_cmd_is_running("reloadModel");
    if is_running == False:
        print("")
        print('Reload Completed!')
        print("")
        break
    time.sleep(5)

# Get the loaded model(s) info...
reload_result = my_notebook.pelion_api.pelion_list_models();
if 'response' in reload_result and len(reload_result['response']) > 0:
    if 'name' in reload_result['response'][0]:
        print("")
        print("Currently Loaded Model(s):")
        print(reload_result)
else:
    print("Nodel: " + model_name + " did NOT load properly. Check sagemaker edge agent logs on GW.")

Now lets do a prediction. We will store the results (with a timestamp) back on S3 so that we can pull it back to our notebook...

In [ ]:
# Invoke the prediction with our input image data
input_data = 's3:///' + input_data_filename_saved
output_result = 's3:///' + model_basename + '-predicted.data'
print("Invoking Prediction on Pelion Edge with Sagemaker. Model: " + model_name + " Input: " + input_data + " Output: " + output_result)
print("")
my_notebook.pelion_api.pelion_predict(model_name, input_data, output_result)

# Poll every 5 sec to look for the predict() completion....
while True:
    print('Predicting...')
    is_running = my_notebook.pelion_api.pelion_cmd_is_running("predict");
    if is_running == False:
        print("")
        print('Prediction Completed!')
        print("")
        break
    time.sleep(5)
    
# Now get the prediction result
prediction_result = my_notebook.pelion_api.pelion_last_cmd_result();
if 'details' in prediction_result:
    if 'output' in prediction_result['details']:
        print("")
        print("Prediction Results:")
        print(prediction_result)

Next we display our results....

In [ ]:
# Prediction results tensor filename in our notebook
prediction_results_tensor_filename = model_basename + '-prediction-output.tensor'

# Copy the results back to our notebook
print("")
print("Retrieving result: " + prediction_result['details']['output'][0]['url'] + " Saving to: " + prediction_results_tensor_filename + "...")
my_notebook.copy_results_to_notebook(prediction_result['details']['output'][0]['url'],prediction_results_tensor_filename)

# Read in the output tensor file, convert it, then decode our predictions and display our results...
print("")
print("Opening Output File: " + prediction_results_tensor_filename + "...")
file_size = os.path.getsize(prediction_results_tensor_filename)
print("Prediction Output File Size: " + str(file_size) + " bytes")
with open(prediction_results_tensor_filename, 'r') as file:
    # Load the JSON-based tensor from its file in our notebook... 
    json_tensor = json.loads(file.read())
    
    # Convert the JSON-based tensor to an Numpy Float32 Tensor with the intended shape
    uint8_buffer = base64.b64decode(json_tensor['b64_data'])
    output_tensor = np.frombuffer(uint8_buffer, dtype=np.float32)
    output_tensor_reshaped = np.reshape(output_tensor,(json_tensor['shape'][0],json_tensor['shape'][1]))
    
    # Display the prediction result tensor details...
    print("")
    print("Output Tensor - Shape: " + json.dumps(output_tensor_reshaped.shape) + " Type: " + str(output_tensor_reshaped.dtype))
    print(output_tensor_reshaped)
    
    print("")
    print("Decoding ResNet50 Imagenet-trained Prediction results...")
    print("")
    most_likely_labels = decode_resnet50_predictions(output_tensor_reshaped, top=1, class_list_path='./model/imagenet_class_index.json')
    my_notebook.display_images(image_data['img'],most_likely_labels)

(NOT) Done!  As a Data Scientist, I could iterate on re-training the model with additional input data, then recompile/deploy/predict to assess training. Here I can run the model within the notebook and get more accurate result:

In [ ]:
# DEBUG - REMOVE
print("")
print("INPUT [Numpy Tensor] - Shape: " + str(nchw_preprocessed_images_list.shape) + " Type: " + str(nchw_preprocessed_images_list.dtype))
print("INPUT [Numpy Tensor] (orig) - Shape: " + str(preprocessed_images_list.shape) + " Type: " + str(preprocessed_images_list.dtype))
test_input_list = nchw_preprocessed_images_list
if channels_first == False:
    # transfer back to channels_last mode
    test_input_list = tf.transpose(nchw_preprocessed_images_list, [0, 2, 3, 1])
    print("INPUT [Numpy Tensor] (reshaped) - Shape: " + str(test_input_list.shape) + " Type: " + str(test_input_list.dtype))
    
direct_prediction_results = resnet50_model.predict(test_input_list)

print("")
print("OUTPUT [Numpy Tensor] - Shape: " + json.dumps(direct_prediction_results.shape) + " Type: " + str(direct_prediction_results.dtype))
print(direct_prediction_results)

print("")
print("Decoding ResNet50 Imagenet-trained Prediction results...")
most_likely_labels = decode_resnet50_predictions(direct_prediction_results, top=1, class_list_path='./model/imagenet_class_index.json')

print("")
print("Displaying prediction results...")
my_notebook.display_images(image_data['img'],most_likely_labels)